In [1]:
suppressMessages(library("CARD", quietly = T))
suppressMessages(library("ggplot2", quietly = T))
suppressMessages(library("reshape2", quietly = T))
suppressMessages(library("gtools", quietly = T))
suppressMessages(library("scatterpie", quietly = T))
suppressMessages(library("glue", quietly = T))

Warning message:
“replacing previous import ‘RcppML::nmf’ by ‘NMF::nmf’ when loading ‘CARD’”


In [2]:
# devtools::install_github('xuranw/MuSiC')

In [3]:
# load spatial data
key = "HFD14"
# subId = "macrophage_"
subId = ""
inputDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDInputs/"

sptPath = glue("{inputDir}{key}_{subId}spt.csv")
spt.df <- read.csv(sptPath)
rownames(spt.df) <- spt.df[,1]
spt.df <- spt.df[,-1]
spt <- as.matrix(spt.df)
dim(spt)

# load spatial coordinates
xyPath = glue("{inputDir}{key}_{subId}coords.csv")
xy.df <- read.csv(xyPath)
rownames(xy.df) <- xy.df[,1]
xy.df <- xy.df[,-1]
dim(xy.df)

# load the rna seq data
rnaPath = glue("{inputDir}{key}_{subId}rna.csv")
rna.df <- read.csv(rnaPath)
rownames(rna.df) <- rna.df[,1]
rna.df <- rna.df[,-1]
rna <- as.matrix(rna.df)
dim(rna)

# load the annotations
metaPath = glue("{inputDir}{key}_{subId}clusters.csv")
meta.df <- read.csv(metaPath)
rownames(meta.df) <- meta.df$cellId
dim(meta.df)

[1] 31053  1994

[1] 1994    2

[1] 31053  6436

[1] 6436    3

In [4]:
CARD_obj = createCARDObject(
	sc_count = rna,
	sc_meta = meta.df,
	spatial_count = spt,
	spatial_location = xy.df,
	ct.varname = "cellType",
	ct.select = unique(meta.df$cellType),
	sample.varname = "sampleInfo",
	minCountGene = 10,
	minCountSpot = 5)

## QC on scRNASeq dataset! ...
## QC on spatially-resolved dataset! ...


In [5]:
CARD_obj = suppressMessages(CARD_deconvolution(CARD_object = CARD_obj))

## create reference matrix from scRNASeq...
## Select Informative Genes! ...
## Deconvolution Starts! ...
## Deconvolution Finish! ...


In [6]:
props.df <- as.data.frame(CARD_obj@Proportion_CARD)

outDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDOuputs/"
outpath = glue("{outDir}{key}_{subId}CARD.csv")

write.csv(props.df, 
          outpath, 
          row.names=TRUE)

dim(props.df)

head(props.df)

[1] 1993    7

,Macrophages,Monocytes,T cells,NK cells,Dendritic cells,Stromal cells,B cells
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAACATTTCCCGGATT.1,0.2731794,0.09297937,0.05971767,0.05473114,0.08580361,0.26020602,0.1733828
AAACCGGGTAGGTACC.1,0.5378256,0.17202652,0.03699356,0.01969402,0.08386505,0.04261869,0.1069765
AAACCGTTCGTCCAGG.1,0.2387206,0.22716739,0.06307571,0.11102764,0.08693683,0.13237798,0.1406939
AAACCTAAGCAGCCGG.1,0.1449351,0.15959771,0.12979579,0.21748579,0.10740280,0.07754814,0.1632347
AAACCTCATGAAGTTG.1,0.3204282,0.19211547,0.06054496,0.11135548,0.04950561,0.10841726,0.1576330
AAACGGGCGTACGGGT.1,0.0703626,0.13902683,0.09680582,0.04934261,0.02241969,0.42272818,0.1993143


In [7]:
# # pieViz <- function(proportion, spatial_location){
# #     res_CARD = as.data.frame(proportion)
# #     res_CARD = res_CARD[,mixedsort(colnames(res_CARD))]
# #     location = as.data.frame(spatial_location) 
# #     colors = colorRampPalette(c("Blue", "Red"))(ncol(res_CARD))
# #     data = cbind(res_CARD,location)
# #     ct.select = colnames(res_CARD)
# #     p = ggplot() + 
# #         geom_scatterpie(aes(x=x, y=y, r = 1), 
# #                         data=data,
# #                         cols=ct.select,color=NA) + 
# #         coord_fixed(ratio = 1) + 
# #         scale_fill_manual(values = colors) 
# #     return(p)
    
# # }

# p <- CARD.visualize.pie(proportion = CARD_obj@Proportion_CARD,
#                         spatial_location = CARD_obj@spatial_location)
                   
# ggsave("cardFigures/mixtures.png",
#        width = 50, 
#        height = 50, 
#        units = "cm")

# print('done')

In [8]:
visCells <- function(proportion, spatial_location, ct.visualize, colors, NumCols, pntSize){
    res_CARD = as.data.frame(proportion)
    res_CARD = res_CARD[,order(colnames(res_CARD))]
    location = as.data.frame(spatial_location)
    if(sum(rownames(res_CARD)==rownames(location))!= nrow(res_CARD)){
    stop("The rownames of proportion data does not match with the rownames of spatial location data")
    }
    ct.select = ct.visualize
    res_CARD = res_CARD[,ct.select]
    res_CARD_scale = as.data.frame(apply(res_CARD,2,function(x){
    (x - min(x)) / (max(x) - min(x))
    } ))
    res_CARD_scale$x = as.numeric(location$x)
    res_CARD_scale$y = as.numeric(location$y)
    mData = melt(res_CARD_scale,id.vars = c("x","y"))
    colnames(mData)[3] <- "Cell_Type"
    b = c(0,1)
    
    p = ggplot(mData, aes(x, y))  +
    geom_point(aes(colour = value), size = pntSize) +
    scale_color_gradientn(colours = colors) + 
    scale_y_reverse() +
    facet_wrap(~Cell_Type,ncol = NumCols)+ 
    coord_fixed()+
    theme(axis.text = element_blank(),
          axis.ticks = element_blank(),
          axis.title = element_blank(),
          panel.background = element_blank(),
          plot.background = element_blank(),
          panel.grid.major = element_blank(), 
          panel.grid.minor = element_blank(),
         )
    return(p)
}

p <- visCells(CARD_obj@Proportion_CARD,
         CARD_obj@spatial_location,
         unique(meta.df$cellType),
         c("lightblue","lightyellow","red"),
         4, 
         0.1)

figDir = "cardFigures/"
outpath = glue("{figDir}{key}_{subId}spots.png")
ggsave(outpath,
       width = 20, 
       height = 20, 
       units = "cm")

print('done')

[1] "done"


In [9]:
CARD_obj = CARD.imputation(CARD_obj,
                           NumGrids = 5000,
                           ineibor = 5,
                           exclude = NULL)

location_imputation = cbind.data.frame(x=as.numeric(sapply(strsplit(rownames(CARD_obj@refined_prop),split="x"),"[",1)),
                                       y=as.numeric(sapply(strsplit(rownames(CARD_obj@refined_prop),split="x"),"[",2)))

rownames(location_imputation) = rownames(CARD_obj@refined_prop)

p <-visCells(
	CARD_obj@refined_prop,                         
	location_imputation,            
	unique(meta.df$cellType),                    
	c("lightblue","lightyellow","red"),    
	NumCols = 4,
    pntSize = 0.1)     

outpath = glue("{figDir}{key}_{subId}imputed.png")
ggsave(outpath,
       width = 20, 
       height = 20, 
       units = "cm")

print('done')

## The rownames of locations are matched ...
## Make grids on new spatial locations ...
[1] "done"


In [10]:
df <- as.data.frame(CARD_obj@refined_prop)
head(df)

head(location_imputation)

,Macrophages,Monocytes,T cells,NK cells,Dendritic cells,Stromal cells,B cells
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
332x480,0.2486352,0.1507260,0.08011425,0.1122454,0.1071986,0.1311844,0.1698962
332x496,0.2485500,0.1507917,0.08009989,0.1122954,0.1072461,0.1311785,0.1698384
332x512,0.2486252,0.1509313,0.08005441,0.1124390,0.1073759,0.1309960,0.1695782
348x512,0.2487715,0.1511788,0.07998653,0.1123467,0.1072129,0.1308608,0.1696428
380x512,0.2479566,0.1511926,0.08018962,0.1125122,0.1073505,0.1310148,0.1697838
396x512,0.2483143,0.1512283,0.08011157,0.1122250,0.1072370,0.1311332,0.1697506


,x,y
,<dbl>,<dbl>
332x480,332,480
332x496,332,496
332x512,332,512
348x512,348,512
380x512,380,512
396x512,396,512


In [11]:
# save the high res file
# head(location_imputation)

props.df.refined <- as.data.frame(CARD_obj@refined_prop)

outDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDOuputs/"
outpath = glue("{outDir}{key}_{subId}CARD_refined.csv")

write.csv(props.df, 
          outpath, 
          row.names=TRUE)

dim(props.df.refined)
head(props.df.refined)

[1] 2637    7

,Macrophages,Monocytes,T cells,NK cells,Dendritic cells,Stromal cells,B cells
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
332x480,0.2486352,0.1507260,0.08011425,0.1122454,0.1071986,0.1311844,0.1698962
332x496,0.2485500,0.1507917,0.08009989,0.1122954,0.1072461,0.1311785,0.1698384
332x512,0.2486252,0.1509313,0.08005441,0.1124390,0.1073759,0.1309960,0.1695782
348x512,0.2487715,0.1511788,0.07998653,0.1123467,0.1072129,0.1308608,0.1696428
380x512,0.2479566,0.1511926,0.08018962,0.1125122,0.1073505,0.1310148,0.1697838
396x512,0.2483143,0.1512283,0.08011157,0.1122250,0.1072370,0.1311332,0.1697506


In [12]:
# pp <- CARD.visualize.gene(
# 	spatial_expression = CARD_obj@refined_expression,
# 	spatial_location = location_imputation,
# 	gene.visualize = c("LPL", "RPL4"),
# 	colors = NULL,
# 	NumCols = 2)

# ggsave("cardFigures/genes.png",
#        width = 20, 
#        height = 20, 
#        units = "cm")

# print('done')

In [13]:
# exp.df <- as.data.frame(rowSums(spt))
# exp.df$names <- rownames(exp.df)
# exp.df.sorted <- exp.df[order(exp.df$rowSums, decreasing = TRUE),]
# head(exp.df.sorted, 20)

In [14]:
# ## select the cell type that we are interested
# ct.visualize = unique(meta.df$cellType)

# ## visualize the spatial distribution of the cell type proportion
# p <- CARD.visualize.prop(
# 	proportion = CARD_obj@Proportion_CARD,        
# 	spatial_location = CARD_obj@spatial_location, 
# 	ct.visualize = ct.visualize,
#     colors = c("lightblue","lightyellow","red"),
#     NumCols=4
# )

# ggsave("test.png", width = 20, height = 20, units = "cm")



# # print(p2)